# Collecte des données : 

In [ ]:
import pandas as pd


# Chargement des fichiers CSV 
traffic_data = pd.read_csv('trafic_reseau.csv')
timestamp_data = pd.read_csv('horodatage du trafic.csv')

# Affichage des premières lignes de chaque fichier pour voir leur structure
traffic_data.head(), timestamp_data.head()

# Calcul de l'entropie :

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Définition de la fonction pour calculer l'entropie de Shannon
def calculate_entropy(data):
    value_counts = data.value_counts()
    probabilities = value_counts / value_counts.sum()
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

# Calcul de l'entropie de Shannon pour chaque seconde dans la colonne 'Destination'
entropy_per_second = traffic_data.groupby('Second')['Destination'].apply(calculate_entropy)

# Stockage des résultats dans un DataFrame
entropy_df = entropy_per_second.reset_index()
entropy_df.columns = ['Second', 'Entropy']

entropy_df.head()


# Création d'un graphe pour visualiser l'entropie par seconde
plt.figure(figsize=(12, 6))
plt.plot(entropy_df['Second'], entropy_df['Entropy'], marker='o', linestyle='-')
plt.title('Entropie de Shannon par Seconde')
plt.xlabel('Second')
plt.ylabel('Entropie')
plt.grid(True)
plt.show()



# Calcul des FTT :

In [ ]:
from scipy.fft import fft
from scipy.signal import find_peaks


# Chargement des données
traffic_data = pd.read_csv('trafic_reseau.csv')

# Fonction pour calculer la FFT des longueurs des paquets
def calculate_fft(data):
    fft_values = np.abs(fft(data))
    return fft_values

# Fonction pour détecter les pics dans la FFT
def find_fft_peaks(data, height_ratio=0.5):
    # Définition du seuil à un pourcentage de l'amplitude du pic le plus élevé
    height_threshold = np.max(data) * height_ratio
    peaks, properties = find_peaks(data, height=height_threshold)
    return peaks, properties['peak_heights']

# Groupement des données par seconde et calcul de la FFT
grouped_data = traffic_data.groupby('Second')['Length']
fft_results = grouped_data.apply(calculate_fft)

# Création d'une liste pour enregistrer le nombre de pics pour chaque seconde
peak_counts = []

# Affichage des graphiques pour chaque seconde avec indication des pics
fig, axes = plt.subplots(nrows=min(5, len(fft_results)), ncols=1, figsize=(10, 20), sharex=True)
for (second, fft_data), ax in zip(fft_results.items(), axes.flatten()):
    peaks, peak_heights = find_fft_peaks(fft_data)
    peak_counts.append((second, len(peaks)))  # Enregistrement du nombre de pics
    ax.plot(fft_data, marker='o', linestyle='-')
    ax.plot(peaks, fft_data[peaks], "x")
    ax.set_title(f'Magnitude de la FFT pour la seconde {second} - {len(peaks)} pics')
    ax.set_xlabel('Fréquence (bins)')
    ax.set_ylabel('Magnitude')

plt.tight_layout()
plt.show()

# Affichage de la liste des secondes avec le nombre de pics
print(peak_counts)


# Fusion des caractéristiques : 

In [ ]:

from scipy.fft import fft
from scipy.signal import find_peaks


# Chargement des données
traffic_data = pd.read_csv('trafic_reseau.csv')

# Calcul de l'entropie de Shannon
def calculate_entropy(data):
    value_counts = data.value_counts()
    probabilities = value_counts / value_counts.sum()
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

# Calcul de la FFT et détection des pics
def calculate_fft(data):
    fft_values = np.abs(fft(data))
    return fft_values

def find_fft_peaks(data, height_ratio=0.5):
    height_threshold = np.max(data) * height_ratio
    peaks, properties = find_peaks(data, height=height_threshold)
    return len(peaks)

# Application des calculs
entropy_results = traffic_data.groupby('Second')['Destination'].apply(calculate_entropy)
fft_results = traffic_data.groupby('Second')['Length'].apply(calculate_fft)
peak_counts = traffic_data.groupby('Second')['Length'].apply(lambda x: find_fft_peaks(calculate_fft(x)))

# Création d'un DataFrame combiné
combined_results = pd.DataFrame({
    'Second': entropy_results.index,
    'Entropy': entropy_results.values,
    'Peak Count': peak_counts.values
})

# Affichage des résultats
print(combined_results.head())


# Normalisation : 

In [ ]:
import pandas as pd




# Supposons que 'combined_results' est le DataFrame contenant les secondes, l'entropie, et le compte de pics
# Calcul des valeurs Min et Max pour l'entropie et le compte de pics
timestamp_data = pd.read_csv('horodatage du trafic.csv')  # Assurez-vous que le nom du fichier est correct

min_entropy = combined_results['Entropy'].min()
max_entropy = combined_results['Entropy'].max()
min_peaks = combined_results['Peak Count'].min()
max_peaks = combined_results['Peak Count'].max()

# Application de la normalisation Min-Max
combined_results['Normalized Entropy'] = (combined_results['Entropy'] - min_entropy) / (max_entropy - min_entropy)
combined_results['Normalized Peak Count'] = (combined_results['Peak Count'] )




combined_results.drop(columns=['Entropy', 'Peak Count'], inplace=True)



# Chargement des données de trafic réseau et de l'horodatage
traffic_data = pd.read_csv('trafic_reseau.csv')
timestamp_data = pd.read_csv('horodatage du trafic.csv')

# Normalisation et préparation des caractéristiques comme décrit précédemment
# Supposons que combined_results contient déjà les caractéristiques normalisées
# Ajout de la colonne 'Attack' depuis timestamp_data
combined_results['Attack'] = timestamp_data['Attack']

# Remplacement des valeurs '-' par '0' pour indiquer l'absence d'attaque et '1' pour une attaque
combined_results['Attack'] = combined_results['Attack'].replace({'-': 0, '1': 1})

# Affichage des données prêtes pour l'entraînement avec la colonne 'Attack' incluse
print(combined_results.head())

# À ce stade, vous pouvez diviser les données en ensembles d'entraînement et de test
# Exemple simple de division en utilisant une méthode basique
from sklearn.model_selection import train_test_split

features = combined_results[['Normalized Entropy', 'Normalized Peak Count']]
labels = combined_results['Attack']
# Sauvegarder le DataFrame 'combined_results' dans un fichier CSV
combined_results.to_csv('combined_results.csv', index=False)




# Préparation des données

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Charger les données à partir du fichier CSV
combined_results = pd.read_csv('combined_results.csv')

# Séparer les caractéristiques et les étiquettes
features = combined_results[['Normalized Entropy', 'Normalized Peak Count']]
labels = combined_results['Attack']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42)




# Modèle XGB

In [ ]:
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Initialiser le classificateur XGBoost
xgb_classifier = xgb.XGBClassifier(random_state=42)

# Entraîner le classificateur XGBoost
xgb_classifier.fit(X_train, y_train)

# Prédire les étiquettes pour les données de test
y_pred_xgb = xgb_classifier.predict(X_test)

# Calcul de la matrice de confusion pour XGBoost
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)

# Affichage de la matrice de confusion sous forme de heatmap pour XGBoost
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_xgb, annot=True, fmt='d', cmap="Blues")
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix (XGBoost)')
plt.show()

# Évaluation des performances du modèle XGBoost avec d'autres métriques
print("Classification Report:")
print(classification_report(y_test, y_pred_xgb))


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import matplotlib.pyplot as plt

# Prédire les probabilités pour les données de test pour le calcul de la courbe ROC
y_pred_proba_xgb = xgb_classifier.predict_proba(X_test)[:, 1]

# Calcul des métriques de performance
accuracy = accuracy_score(y_test, y_pred_xgb)  # Correction: ajout de cette ligne pour calculer l'exactitude
precision = precision_score(y_test, y_pred_xgb, average='binary')  # Assurez-vous de définir l'argument 'average' correctement
recall = recall_score(y_test, y_pred_xgb, average='binary')
f1 = f1_score(y_test, y_pred_xgb, average='binary')

# Calcul de la courbe ROC et de l'aire sous la courbe (AUC)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_xgb)  # Utilisation de probabilités plutôt que des étiquettes
roc_auc = auc(fpr, tpr)

# Affichage des métriques
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)

# Affichage de la courbe ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()
